In [8]:
! pip install import-ipynb

In [35]:
import pandas as pd
import import_ipynb
from create_pswm import get_pswm
import math

In [36]:
def validation_testing(matrix , df , index): #index è il numero del training set da prendere come validation
    df_val = df.query("Set == @index") #take only validation sequences

    # iterate among all sequences in the validation set. 

    validation_scores = [] #list that contains all the scores of the sequences
    
    for _ , row in df_val.iterrows():
        
        seq = row["Sequence"] #extract the sequence from the dataframe
        pos = 0 # counter for window size tracking
        aa = 0 # index that moves along the sequence
        tmp_score = 0 #inizialise window score
        max_window_score = -math.inf , 0 , 0 , "seq" # initialize a variable that keeps in memory only the best window 
        
        end = min(90, len(seq)) # we end iterating on the sequence when we reach the 90th aa or when the sequence ends (sequence length < 90 aa)
        
        while aa < end - 14: # iteration on the sequence
            while pos <= 14: # iteration while the window is less than the fixed window size
                tmp_score += matrix[seq[aa + pos]][pos] # computing the score of the window
                pos +=1 # increasing the window size counter

            if tmp_score > max_window_score[0]: # checking if the actual window has a better score than the previous one
                max_window_score = tmp_score, aa, aa + pos, seq[aa:aa + pos]

            aa += 1 # increase the starting point
            pos = 0 # resetting the window size
                
        
        validation_scores.append(max_window_score) #da trasformare in dataframe usando corrispondenza indici lista con indici df_val con sequenze

    return validation_scores

In [37]:
if __name__ == "__main__":
    df=pd.read_csv("../Data_Preparation/train_bench.tsv", sep="\t")
    df_pos=df.query("Class=='Positive' and (Set=='1' or Set=='2' or Set=='3' or Set=='4' or Set=='5')").reset_index()
    for index,row in df_pos.iterrows():
            sequence_to_recover=row["Sequence"]
            cleavage_position=int(row["SPEnd"])
            df_pos.loc[index, "Sequence"] = sequence_to_recover[cleavage_position-13:cleavage_position+2]

    matrix = get_pswm(df_pos)
    print(validation_testing(matrix, df, "1"))

[(12.878842422588168, 12, 27, 'AVLILVHESNAQCRR'), (-4.169394820387013, 17, 32, 'ALLLALGVERALALP'), (29.587909104340376, 11, 26, 'FLTGCQAWEFWQQDE'), (23.541022718976443, 17, 32, 'PLLGLVGASGPRTLV'), (-12.401267499644124, 0, 15, 'MEKSTMSAILLVLHL'), (37.38810568149048, 14, 29, 'LGVLVAAPDGGFDLS'), (-7.099728422767681, 0, 15, 'MKTKIIIYICIWATA'), (-5.032682633750392, 0, 15, 'MKRASAGGSRLLAWV'), (25.60998429471715, 15, 30, 'ILPGTLCAEGTRGRS'), (21.863947862648097, 8, 23, 'LFVLGTVIRADQCPP'), (-0.7972238720773672, 1, 16, 'QLTRCCFVFLVQGSL'), (49.98759335107245, 11, 26, 'ALVAAAWGAQAPRTP'), (-7.250432955617641, 13, 28, 'MAMLWVTPVRAGTEC'), (34.23676035805401, 12, 27, 'ALLVLPEATKAEEGS'), (-8.024439443954492, 0, 15, 'MTSIFHFAIIFMLIL'), (27.65446308065995, 7, 22, 'AVLLVAQPQLVHSHP'), (31.047738952077562, 15, 30, 'SVLAAPSPLDARATW'), (35.429255166686914, 6, 21, 'MVVVLALSSAQDPNR'), (-6.472403230360844, 0, 15, 'MEGVLLFWKYRALFF'), (8.471313726552868, 21, 36, 'PPALLSLGVDAQPLP'), (-8.168246186131926, 0, 15, 'MNK